# Introduction to Prompt Engineering

*Prompt* เป็นการป้อนคำสั่งให้กับ large language model (LLM) เช่น GPT, Gemini, หรือ Llama เพื่อให้ได้มาซึ่งผลลัพธ์ที่ต้องการ ซึ่งต่อมาได้มีการพัฒนาเทคนิคต่างๆ ในการเขียน prompt ที่ทำให้ผลลัพธ์ที่ได้มีคุณภาพมากขึ้น ซึ่งเราเรียกเทคนิคเหล่านั้นว่า *prompt engineering*

ใน Notebook นี้เราจะยกตัวอย่างการเขียน prompt และ เปรียบเทียบผลลัพธ์ที่ได้จาก Prompt และ โมเดลที่แตกต่างกัน
โดยใช้ *LlamaIndex* ซึ่งเป็น เป็นเฟรมเวิร์กข้อมูลที่เรียบง่ายและยืดหยุ่นสำหรับเชื่อมต่อแหล่งข้อมูลที่กำหนดเองกับ LLM ที่เราต้องการ 

In [ ]:
# Reference: https://platform.openai.com/docs/guides/vision
import io
import base64
from PIL import Image
from openai import OpenAI
from IPython.display import display

OPENAI_API_KEY = ""
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_completion(prompt: str, model: str = "gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

get_completion("Why is sky blue?")

In [ ]:
response = get_completion("What is the meaning of life?")
print(response)

## **Prompt Engineering**

In [ ]:
# TODOs

## **Multimodal with ChatGPT**

In [ ]:
# Load the image from a path
image_path = 'assets/seki_example.jpg'
image = Image.open(image_path)
display(image)

In [ ]:
system_prompt = """\
You are a helpful assistant designed to extract information from the input document and user question.\
Please always answer the question based on the information extracted from the document and in a concise manner.
"""

In [ ]:
def encode_image(image: Image.Image) -> str:
    """Encode an image into base64 format."""
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def create_link(base64_image: str) -> str:
    """Create a link from a base64 image."""
    return f"data:image/jpeg;base64,{base64_image}"

def extract_information(image_path: str, prompt: str) -> str:
    """Extract information from the image and return the answer."""
    # Load the image
    image = Image.open(image_path)
    # Encode the image
    encoded_image = encode_image(image)
    image_link = create_link(encoded_image)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_link,
                        }
                    }
                ]
            }
        ],
        response_format={ "type": "json_object" } # response in JSON format!
    )

    answer = response.choices[0].message.content
    return answer

In [ ]:
question = """
Perform optical character information and export in the following JSON format.

{
  tax_id: str,
  pos_id: str,
  tel_number: str,
  date: str,
  recepit_number: str,
  items: list[name, float],
  price_before_vat: float,
  total_price: float,
  vat_7_percent: float,
  earn_point: float
}
"""
answer = extract_information(image_path, question)
print("Answer:", answer)

## **Explain Nong Moo Deng**

ลองใส่ภาพน้องหมูเด้งแล้วทดลองใช้ ChatGPT อธิบายว่าทำไมน้องถึงเป็น Viral ในโลกออนไลน์

In [ ]:
system_prompt = """\
You are a helpful assistant designed to interpret the image and see why the given image gets viral online. You should provide an answer in Thai.
"""
query = "Give an image, please explain why the given image of 'หมูเด้ง', a baby hippo, is viral online?"

In [ ]:
def generate_output(image_path: str, prompt: str) -> str:
    """Extract information from the image and return the answer."""
    # Load the image
    image = Image.open(image_path)
    # Encode the image
    encoded_image = encode_image(image)
    image_link = create_link(encoded_image)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_link,
                        }
                    }
                ]
            }
        ],
    )

    answer = response.choices[0].message.content
    return answer

In [ ]:
image_path = "assets/moodeng.jpg"
output = generate_output(image_path, query)
print(output)